In [1]:
import pyterrier as pt
pt.init(tqdm='notebook')

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
import torch
torch.__version__

'1.10.0+cu113'

## load the index

In [3]:
import pyterrier_colbert
from pyterrier_colbert.ranking import *
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"
index=("/nfs/indices/colbert_passage","index_name3")
factory = pyterrier_colbert.ranking.ColBERTFactory(checkpoint, *index, )

# from pyterrier_colbert.ranking import ColBERTFactory
# factory = ColBERTFactory.from_dataset('vaswani', 'colbert_uog44k')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Feb 10, 19:01:34] #> Loading model checkpoint.
[Feb 10, 19:01:34] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


/opt/conda/envs/colbert38/lib/python3.8/site-packages/torch/hub.py:513: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[Feb 10, 19:01:43] #> checkpoint['epoch'] = 0
[Feb 10, 19:01:43] #> checkpoint['batch'] = 44500


In [4]:
factory.faiss_index_on_gpu = False
nn_term = factory.nn_term(df=True)

[Feb 10, 19:01:44] #> Loading the FAISS index from /nfs/indices/colbert_passage/index_name3/ivfpq.faiss ..
[Feb 10, 19:01:55] #> Building the emb2pid mapping..
[Feb 10, 19:02:17] len(self.emb2pid) = 687989391
[Feb 10, 19:02:23] #> Building the emb2tid mapping..
687989391
Loading doclens


In [5]:
[x.mmap.shape[0] for x in factory._rrm().part_mmap]

Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/24 [00:00<?, ?shard/s]

[28603376,
 28935602,
 28997615,
 29001911,
 29033097,
 29094152,
 28649536,
 28723825,
 28856902,
 28970728,
 29018059,
 29025293,
 29029230,
 29065641,
 29474976,
 30061199,
 33096705,
 30056847,
 29293124,
 27428802,
 28026855,
 27215683,
 28375647,
 19954586]

## index stats

In [6]:
total = torch.sum(nn_term.lookup)
total

tensor(687989391)

In [7]:
len(factory)

8841823

In [8]:
total/len(factory)

tensor(77.8108)

## eval setup

In [ ]:
%pip install --force-reinstall --no-deps git+https://github.com/terrierteam/ir_measures.git@runtime

In [9]:
def _AvgDocLen(qrels, run):
    from warnings import warn
    if 'doc_toks' not in run.columns:
        if len(run) > 0:
            warn("run of %d rows did not have doc_toks column; available columns: %s" % (len(run), str(run.columns)))
        else:
            warn("empty run did not have doc_toks column; available columns: %s" % str(run.columns))
        return 0
    return run['doc_toks'].apply(lambda row: row.shape[0]).mean()
    
import ir_measures
AvgDocLen = ir_measures.define_byquery(
    _AvgDocLen, 
    name="AvgDoclen")

from pyterrier.measures import *
MEASURES = [AP(rel=2)@1000, nDCG@10,nDCG@20,nDCG@100, RR(rel=2)@10,RR(rel=2),RR@10, RR ,R(rel=2)@1000,"mrt", AvgDocLen@1, AvgDocLen@10, AvgDocLen@100, AvgDocLen]


In [10]:
def do_exp(pipelines, topics='2019', names=None, baseline=None, save_dir=None) -> pd.DataFrame:
    TQ = None
    topics_map = { '2019' : 'test-2019', '2020' : 'test-2020', 'dev' : 'dev.small' }
    TQ = (
       pt.get_dataset("msmarco_passage").get_topics(topics_map[topics]), 
       pt.get_dataset("msmarco_passage").get_qrels(topics_map[topics])
    )
    if save_dir is not None:
        save_dir = save_dir % str(topics)
        os.makedirs(save_dir, exist_ok=True)
    df = pt.Experiment(
        pipelines, 
        *TQ,
        MEASURES,
        batch_size=20,
        filter_by_qrels=True,
        verbose=True,
        names=names,
        save_dir=save_dir
    )
    if save_dir:
        df.to_pickle(save_dir + "/results_df.pkl")
        
    return df

## stops uniform pruning

In [11]:
if not os.path.exists("stopword-list.txt"):
    !wget "https://raw.githubusercontent.com/terrier-org/terrier-core/5.x/modules/core/src/main/resources/stopword-list.txt"

stops=[]
with open("stopword-list.txt") as f:
    for l in f:
        stops.append(l.strip())

In [12]:
def uniform_blacklisted_tokens_transformer(blacklist, verbose=False, zero=False, gpu=torch.cuda.is_available()) -> TransformerBase:
    """
    Remove tokens and their embeddings from the document dataframe
    input: qid, query_embs, docno, doc_embs, doc_toks
    output: qid, query_embs, docno, doc_embs, doc_toks
    
    The blacklist parameters must contain a list of tokenids that should be removed
    """
    import pyterrier as pt
    import torch
    import numpy as np
    
    assert pt.started(), 'PyTerrier must be started'
    
    if gpu: 
        blacklist = torch.Tensor(blacklist).cuda()
    else:
        blacklist = torch.Tensor(blacklist)
        
    pt.tqdm.pandas()
    
    if verbose: print(f'Blacklist composed of {len(blacklist)} elements.')
        
    def _prune_remove_gpu(row):
        tokens = row['doc_toks'].cuda()
        embeddings = row['doc_embs'].cuda()

        # create the 1-D mask
        final_mask = torch.all(tokens[None, : ] != blacklist[:, None], axis=0)

        # apply the mask to the tokens
        row['doc_toks'] = tokens[final_mask]

        # pad the 1-d mask
        row_embs_size = embeddings.size()
        mask_1d = torch.cat((final_mask, torch.ones(row_embs_size[0] - final_mask.size()[0], dtype=torch.bool).cuda()))

        # apply the padded 1-d mask
        row['doc_embs'] = embeddings[mask_1d, : ]
        return row

    def _prune_remove(row):
        tokens = row['doc_toks']
        embeddings = row['doc_embs']

        # create the 1-D mask
        final_mask = torch.all(tokens[None, : ] != blacklist[:, None], axis=0)

        # apply the mask to the tokens
        row['doc_toks'] = tokens[final_mask]

        # pad the 1-d mask
        row_embs_size = embeddings.size()
        mask_1d = torch.cat((final_mask, torch.ones(row_embs_size[0] - final_mask.size()[0], dtype=torch.bool)))

        # apply the padded 1-d mask
        row['doc_embs'] = embeddings[mask_1d, : ]
        return row
    
    def _prune_gpu_zero(row):
        tokens = row['doc_toks'].cuda()
        embeddings = row['doc_embs'].cuda()
        row_embs_size = embeddings.size()
        tokens_size = tokens.size()[0]
        
        # create the 1-D mask
        final_mask = torch.zeros(row_embs_size[0], dtype=torch.bool)
        final_mask[0:tokens_size] = torch.any(tokens[None, :] == blacklist[:, None], axis=0)
        
        # apply the mask
        row['doc_toks'][final_mask[0:tokens_size]] = 0
        row['doc_embs'][final_mask, :] = 0 
        return row
        
    def _prune_zero(row):
        tokens = row['doc_toks']
        embeddings = row['doc_embs']
        row_embs_size = embeddings.size()
        tokens_size = tokens.size()[0]
                                        
        # create the 1-D mask
        final_mask = torch.zeros(row_embs_size[0], dtype=torch.bool)
        final_mask[0:tokens_size] = torch.any(tokens[None, :] == blacklist[:, None], axis=0)
            
        # apply the mask
        row['doc_toks'][final_mask[0:tokens_size]] = 0
        row['doc_embs'][final_mask, :] = 0
        return row
    
    if zero:
        prune_function = _prune_gpu_zero if gpu else _prune_zero
    else:
        prune_function = _prune_remove_gpu if gpu else _prune_remove

    def _apply(df):
        if verbose:
            df = df.progress_apply(prune_function, axis=1)
        else:
            df = df.apply(prune_function, axis=1)
        return df

    return pt.apply.generic(_apply)

def get_uniform_stop_pipeline(factory, blacklist, k1=1000):
    pipeline = (
        factory.query_encoder()
        >> (factory.ann_retrieve_score(query_encoded=True) % k1)
        >> factory.fetch_index_encodings(ids=True, verbose=False)
        >> uniform_blacklisted_tokens_transformer(blacklist, verbose=False, zero=False)
        >> factory.scorer(verbose=False)
        #>> pt.apply.doc_embs(drop=True)
        #>> pt.apply.query_embs(drop=True)
    )
    return pipeline

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
stop_ids = [x for x in tokenizer.convert_tokens_to_ids(stops) if x != 100]

In [14]:
stops_2019 = do_exp(
    [get_uniform_stop_pipeline(factory, stop_ids)], 
    topics='2019',
    names=["stops_uniform" ],
    save_dir="/nfs/craigm/staticPruning/results/repro/Cstops_uniform_%s/"
)
stops_2019

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

pt.Experiment:   0%|          | 0/3 [00:00<?, ?batches/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen,mrt
0,stops_uniform,0.398659,0.687096,0.638232,0.572497,0.830233,0.83041,0.928295,0.928295,0.695251,45.581395,52.155814,53.665116,52.245628,657.402888


In [15]:
stops_2019 = do_exp(
    [get_uniform_stop_pipeline(factory, stop_ids)], 
    topics='2020',
    names=["stops_uniform" ],
    save_dir="/nfs/craigm/staticPruning/results/repro/Cstops_uniform_%s/"
)
stops_2019

msmarco-test2020-queries.tsv.gz:   0%|          | 0.00/4.03k [00:00<?, ?iB/s]

2020qrels-docs.txt:   0%|          | 0.00/213k [00:00<?, ?iB/s]

pt.Experiment:   0%|          | 0/3 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,stops_uniform,0.454838,0.687692,0.653287,0.606888,0.86034,0.861764,0.916667,0.918091,0.754593,45.407407,49.985185,52.892407,51.988426


In [18]:
stops_dev = do_exp(
    [get_uniform_stop_pipeline(factory, stop_ids)],
    topics='dev',
    names=["stops_uniform" ],
    save_dir="/nfs/staticPruning/results/repro/Cstops_uniform_%s/"
)
stops_dev

pt.Experiment:   0%|          | 0/349 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,stops_uniform,0.0,0.391132,0.416073,0.444741,0.0,0.0,0.331263,0.342257,0.0,47.682235,51.46616,53.279136,51.901785


## random document-centrics

In [21]:

from typing import Tuple, Sequence


def doc_centric_random(tokenids : torch.Tensor, embs : torch.Tensor, num_prune, rng) ->  Tuple[Sequence[int], torch.Tensor]:
    """
    Returns the tokenids and embs after remvoing num_prune embeddings
    """
    
    length = len(tokenids)
    
    newmask = torch.BoolTensor(length)
    newmask[:] = True
    embs = embs[0:length, :]   
    
    for i in rng.sample(range(length), min(length, num_prune)):
        newmask[i] = False
        
    return tokenids[newmask], embs[newmask,:]


def prune_doccentric_transformerH0(factory, topK, active_ids=None, min_obs=0, seed=42):
    import random
    
    rng = random.Random(seed)
    assert active_ids is None 
    
    def _prune(df):
        #pt.tqdm.pandas(desc='prune_doccentric')
        #progress_apply
        df[["doc_toks", "doc_embs"]] = df.apply(
            lambda row: doc_centric_random(row.doc_toks, row.doc_embs, topK, rng),
            #lambda row: tc.ignore_prune(row.doc_toks, row.doc_embs, topK, pruning='global', blacklist=blacklist), 
            result_type="expand", 
            axis=1)
        return df
    return pt.apply.generic(_prune)

def prune_doccentric_random_pipeH0(topK, approx=True):
    first_pass = (factory.ann_retrieve_score() % 1000) if approx else factory.set_retrieve()
    return ( first_pass
        >> factory.fetch_index_encodings(ids=True, verbose=False) 
        >> prune_doccentric_transformerH0(factory, topK)
        >> factory.scorer(verbose=False))


In [22]:
prune_doccentric_random_pipeH0(10).search("chemical reactions")[['qid', 'docno']]

,qid,docno
1537,1,5453527
1999,1,7605154
665,1,1833253
664,1,1833252
586,1,1525431
...,...,...
1533,1,5436251
2164,1,8311152
2053,1,7875435
1023,1,3444362


In [23]:
DOC_CENTRIC_THRESHOLDS=[1,3,5,8,10,15,20,25,30]

In [24]:
doccentric_2019 = do_exp(
    [prune_doccentric_random_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='2019',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/random_doccentric_%s/"
)
doccentric_2019

pt.Experiment:   0%|          | 0/27 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,doc_centric_1,0.406474,0.694532,0.650674,0.584214,0.853488,0.853659,0.939922,0.939922,0.695251,67.255814,75.909302,78.472326,75.997488
1,doc_centric_3,0.403502,0.695778,0.654692,0.584474,0.865116,0.865290,0.951550,0.951550,0.695251,64.976744,75.209302,76.796047,73.997488
2,doc_centric_5,0.395113,0.678131,0.646355,0.577801,0.841860,0.842045,0.943798,0.943798,0.695251,68.372093,73.232558,75.153256,71.997488
3,doc_centric_8,0.390897,0.676074,0.644773,0.578178,0.818605,0.818789,0.934109,0.934109,0.695251,62.162791,70.541860,72.909767,68.997488
4,doc_centric_10,0.386604,0.670099,0.634522,0.571584,0.860465,0.860656,0.939922,0.939922,0.695251,65.441860,69.848837,71.484186,66.997488
5,doc_centric_15,0.376311,0.668657,0.632718,0.564782,0.818798,0.818999,0.959302,0.959302,0.695251,58.976744,66.506977,67.436279,61.997581
6,doc_centric_20,0.369363,0.651749,0.611360,0.547354,0.846899,0.847103,0.937984,0.937984,0.695251,57.813953,63.383721,64.293256,56.999023
7,doc_centric_25,0.347077,0.635784,0.601294,0.538971,0.854651,0.854831,0.934109,0.934109,0.695251,57.930233,61.118605,61.310930,52.009907
8,doc_centric_30,0.340047,0.623785,0.586891,0.521498,0.825581,0.825834,0.914729,0.914729,0.695251,53.906977,59.611628,58.687674,47.047233


In [25]:
doccentric_2020 = do_exp(
    [prune_doccentric_random_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='2020',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/random_doccentric_%s/"
)
doccentric_2020

pt.Experiment:   0%|          | 0/27 [00:00<?, ?batches/s]

,name,AP(rel=2)@1000,nDCG@10,nDCG@20,nDCG@100,RR(rel=2)@10,RR(rel=2),RR@10,RR,R(rel=2)@1000,AvgDoclen@1,AvgDoclen@10,AvgDoclen@100,AvgDoclen
0,doc_centric_1,0.456601,0.685930,0.655573,0.612020,0.831481,0.832997,0.922840,0.924399,0.754593,69.037037,73.529630,77.506481,75.977167
1,doc_centric_3,0.457122,0.688187,0.657075,0.611438,0.843827,0.846212,0.922840,0.925224,0.754593,67.648148,72.335185,75.827778,73.977167
2,doc_centric_5,0.447944,0.685629,0.649351,0.606512,0.814991,0.816314,0.916049,0.917372,0.754593,65.574074,71.533333,74.315926,71.977167
3,doc_centric_8,0.456141,0.693776,0.653468,0.605945,0.866049,0.866524,0.933951,0.934425,0.754593,64.129630,69.527778,72.097593,68.977167
4,doc_centric_10,0.435066,0.677901,0.638176,0.594374,0.813007,0.814691,0.920370,0.922054,0.754593,63.074074,68.255556,70.834444,66.977167
5,doc_centric_15,0.431837,0.671697,0.635915,0.592673,0.823045,0.824729,0.929527,0.931210,0.754593,57.500000,66.344444,67.368519,61.977352
6,doc_centric_20,0.433849,0.671999,0.634266,0.586405,0.831349,0.831349,0.925485,0.925485,0.754593,55.000000,62.372222,63.981667,56.981333
7,doc_centric_25,0.412176,0.653189,0.618099,0.570496,0.854034,0.854034,0.948942,0.948942,0.754593,53.981481,60.385185,61.201481,52.002389
8,doc_centric_30,0.384685,0.638039,0.599165,0.551604,0.839506,0.839506,0.950617,0.950617,0.754593,50.500000,58.340741,58.346481,47.050130


In [ ]:
doccentric_dev = do_exp(
    [prune_doccentric_random_pipeH0(x) for x in DOC_CENTRIC_THRESHOLDS], 
    topics='dev',
    names=["doc_centric_%d" % x for x in DOC_CENTRIC_THRESHOLDS],
    save_dir="/nfs/staticPruning/results/repro/random_doccentric_%s/"
)
doccentric_dev

pt.Experiment:   0%|          | 0/3141 [00:00<?, ?batches/s]